In [9]:
import os
import pprint
import numpy as np
import riskslim

data_csv_file = 'data_train.csv'
sample_weights_csv_file = None                              # csv file of sample weights for the dataset (optional)

# problem parameters
max_coefficient = 5                                         # value of largest/smallest coefficient
max_L0_value = 5                                            # maximum model size (set as float(inf))
max_offset = 50                                             # maximum value of offset parameter (optional)
c0_value = 1e-6                                             # L0-penalty parameter such that c0_value > 0; larger values -> sparser models; we set to a small value (1e-6) so that we get a model with      max_L0_value terms


In [10]:
# load data from disk
data = riskslim.load_data_from_csv(dataset_csv_file = data_csv_file, sample_weights_csv_file = sample_weights_csv_file)

# create coefficient set and set the value of the offset parameter
coef_set = riskslim.CoefficientSet(variable_names = data['variable_names'], lb = -max_coefficient, ub = max_coefficient, sign = 0)
coef_set.update_intercept_bounds(X = data['X'], y = data['Y'], max_offset = max_offset)

constraints = {
    'L0_min': 0,
    'L0_max': max_L0_value,
    'coef_set':coef_set,
}
 
# major settings (see riskslim_ex_02_complete for full set of options)
settings = {
    # Problem Parameters
    'c0_value': c0_value,
    #
    # LCPA Settings
    'max_runtime': 30.0,                               # max runtime for LCPA
    'max_tolerance': np.finfo('float').eps,             # tolerance to stop LCPA (set to 0 to return provably optimal solution)
    'display_cplex_progress': True,                     # print CPLEX progress on screen
    'loss_computation': 'fast',                         # how to compute the loss function ('normal','fast','lookup')
    #
    # LCPA Improvements
    'round_flag': True,                                # round continuous solutions with SeqRd
    'polish_flag': True,                               # polish integer feasible solutions with DCD
    'chained_updates_flag': True,                      # use chained updates
    'add_cuts_at_heuristic_solutions': True,            # add cuts at integer feasible solutions found using polishing/rounding
    #
    # Initialization
    'initialization_flag': True,                       # use initialization procedure
    'init_max_runtime': 120.0,                         # max time to run CPA in initialization procedure
    'init_max_coefficient_gap': 0.49,
    #
    # CPLEX Solver Parameters
    'cplex_randomseed': 0,                              # random seed
    'cplex_mipemphasis': 0,                             # cplex MIP strategy
}

# train model using lattice_cpa
model_info, mip_info, lcpa_info = riskslim.run_lattice_cpa(data, constraints, settings)

#print model contains model
riskslim.print_model(model_info['solution'], data)

#model info contains key results
pprint.pprint(model_info)

setting c0_value = 0.0 for (Intercept) to ensure that intercept is not penalized
05/24/22 @ 11:20 PM | switching loss computation from fast to lookup
05/24/22 @ 11:20 PM | 201 rows in lookup table
05/24/22 @ 11:20 PM | ------------------------------------------------------------
05/24/22 @ 11:20 PM | runnning initialization procedure
05/24/22 @ 11:20 PM | ------------------------------------------------------------
05/24/22 @ 11:20 PM | CPA produced 2 cuts
05/24/22 @ 11:20 PM | running naive rounding on 6 solutions
05/24/22 @ 11:20 PM | best objective value: 0.4002
05/24/22 @ 11:20 PM | rounding produced 5 integer solutions
05/24/22 @ 11:20 PM | best objective value is 0.3711
05/24/22 @ 11:20 PM | running sequential rounding on 6 solutions
05/24/22 @ 11:20 PM | best objective value: 0.4002
05/24/22 @ 11:20 PM | sequential rounding produced 5 integer solutions
05/24/22 @ 11:20 PM | best objective value: 0.3373
05/24/22 @ 11:20 PM | polishing 10 solutions
05/24/22 @ 11:20 PM | best objec

Lazy constraint(s) or lazy constraint/branch callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling presolve reductions that prevent crushing forms (CPX_PARAM_PREREFORM).
         Disabling repeat represolve because of lazy constraint/incumbent callback.
Tried aggregator 1 time.
Reduced MIP has 22 rows, 24 columns, and 63 nonzeros.
Reduced MIP has 10 binaries, 12 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)
05/24/22 @ 11:20 PM | adding 218 initial cuts

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variable B NodeID Parent  Depth

      0     0        0.2045    21                      0.2045  

In [11]:
import numpy as np

def predict_df(X, rho):
    rho_values = np.copy(rho)
    
    if len(rho) > len(X.columns):
        intercept = rho[0]
        rho_values = np.delete(rho_values, 0)
    else:
        intercept = 0
        
    scores = np.dot(X.values, rho_values)
    probs = 1/(1+np.exp(-(intercept + scores)))
    predictions = (probs >= 0.5).astype(int)
    
    return scores, probs, predictions


In [12]:
import pandas as pd

data_test = pd.read_csv('data_test.csv')

X_test = data_test.drop(['hosdeath_flag'], axis=1)
y_test = data_test['hosdeath_flag']

In [13]:
scores, probs, predictions = predict_df(X_test, model_info['solution'])

In [14]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

roc_auc = roc_auc_score(y_test, probs)
print("ROC AUC: " + str(roc_auc))

average_precision = average_precision_score(y_test, probs)
print("Average Precision: " + str(average_precision))

# accuracy = accuracy_score(y_test, predictions)
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: " + str(accuracy))

# print(confusion_matrix(y_test, predictions))
print(confusion_matrix(y_test, predictions))


ROC AUC: 0.8143934231965635
Average Precision: 0.22206481986727808
Accuracy: 0.9215431296055483
[[2103   47]
 [ 134   23]]


In [15]:
import numpy as np
import scipy.stats


def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, m-h, m+h


In [16]:
from sklearn.utils import resample

auroc_array = []
auprc_array = []
accuracy_array = []

for i in range(50):
    X_test_bs = resample(X_test, n_samples=200, replace=False, stratify=y_test,
             random_state=i)
    y_test_bs = y_test[X_test_bs.index]

    scores,probs,predictions = predict_df(X_test_bs, model_info['solution'])
    auroc = roc_auc_score(y_test_bs, probs)
    auprc = average_precision_score(y_test_bs, probs)
    accuracy = accuracy_score(y_test_bs, predictions)
    
    auroc_array.append(auroc)
    auprc_array.append(auprc)
    accuracy_array.append(accuracy)
    

print(mean_confidence_interval(auroc_array))
print(mean_confidence_interval(auprc_array))
print(mean_confidence_interval(accuracy_array))

(0.8011559139784946, 0.7851012321477291, 0.8172105958092601)
(0.22845801567709462, 0.2074683059157052, 0.24944772543848404)
(0.9174999999999999, 0.9142488656487969, 0.9207511343512028)
